# Paddy Disease Classification

In this Notebook I would try to train a classification model with fastai while at the same time keep track of experiments with Weight and Biases.  
This is strongly based on the Walkthrus sessions by Jeremy Howard in the context of the 2022 fastai course.



## References
- [A bunch of experiments on integrating `fastai` with `timm`](https://github.com/tcapelle/fastai_timm#a-bunch-of-experiments-on-integrating-fastai-with-timm)

In [2]:
import os
from pathlib import Path
import zipfile
import kaggle

from fastai.vision.all import *
import timm

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

kaggle_json_path = Path('./keys/kaggle.json')

with open(kaggle_json_path) as f:
    kaggle_creds = f.read()
    
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
cred_path.parent.mkdir(exist_ok=True)
cred_path.write_text(kaggle_creds)
cred_path.chmod(0o600)

In [3]:
path = Path('paddy-disease-classification')

train_path = path/'train_images'
train_files = get_image_files(train_path)

test_path = path/'test_images'
test_files = get_image_files(train_path).sorted()

In [4]:
if not iskaggle and not path.exists():
    ! kaggle competitions download -c paddy-disease-classification
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [5]:
tta_res = []

In [11]:
def train(epochs, lr, desc, arch, item_tfms, batch_tfms, accum=False):
    
    kwargs = {'bs': 32} if accum else {}
    dls = ImageDataLoaders.from_folder(
        train_path,
        #seed=42,
        valid_pct=0.2,
        item_tfms=item_tfms,
        batch_tfms=batch_tfms,
        **kwargs
    )
    cbs = GradientAccumulation(2) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    learn.fine_tune(epochs, lr)
    tta_res.append(learn.tta(dl=dls.test_dl(test_files)))
    
    learn.export(f'{arch}_{desc}')
    
## For walkthru 12
def custom_train(epochs, lr, arch, item_tfms, batch_tfms, seed=None, accum=False):
    kwargs = {'bs': 32} if accum else {}
    dls = ImageDataLoaders.from_folder(
        train_path,
        seed=seed,
        valid_pct=0.2,
        item_tfms=item_tfms,
        batch_tfms=batch_tfms,
        **kwargs
    )
    # 2 was a misconception, changed to 64 (Walkthru 14 or 15)
    #cbs = GradientAccumulation(2) if accum else []
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    learn.fine_tune(epochs, lr)
    
    return learn

In [12]:
arch = 'swinv2_base_window12_192_22k'
#arch = 'convnext_large_in22k'

In [13]:
learn = custom_train(
    epochs=4,
    lr=0.01,
    arch=arch,
    item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=192, min_scale=0.75),
    seed=42,
    accum=True,
    
)





epoch,train_loss,valid_loss,error_rate,time


KeyboardInterrupt: 

In [8]:
learn.export('swinv2_b_w12_192_22k-4epochs-lr01.pkl')